# Quickstart

This is a brief introduction to motleycrew. 

For a working example of agents, tools, crew, and SimpleTask, check out the [blog with images](examples/blog_with_images.html).

For a working example of custom tasks that fully utilize the knowledge graph backend, check out the [research agent](examples/research_agent.html).


## Agents and tools

Motleycrew provides thin wrappers for all the common agent frameworks: Langchain, LlamaIndex, CrewAI, and Autogen (please let us know if you want any others added!).
It also provides thin wrappers for Langchain and LlamaIndex tools, allowing you to use any of these tools with any of these agents.

Motleycrew also supports **delegation**: you can simply give any agent as a tool to any other agent. 

All the wrappers for tools and agents implement the Runnable interface, so you can use them as-is in LCEL and Langgraph code.

## Crew and tasks

The other two key concepts in motleycrew are crew and tasks. The crew is the orchestrator for tasks, and must be passed to all tasks at creation; tasks can be connected into a DAG using the `>>` operator, for example `TaskA >> TaskB`. This means that `TaskB` will not be started before `TaskA` is complete, and will be given `TaskA`'s output.

Once all tasks and their relationships have been set up, it all can be run via `crew.run()`, which returns a list of the executed `TaskUnits` (see below for details).

### SimpleTask
`SimpleTask` is a basic implementation of the `Task` API. It only requires a crew, a description, and an agent. When it's executed, the description is combined with the output of any upstream tasks and passed on to the agent, and the agent's output is the tasks's output. 

For a working illustration of all the concepts so far, see the [blog with images](examples/blog_with_images.html) example.

## Knowledge graph backend and custom tasks

The functionality so far is convenient, allowing us to mix all the popular agents and tools, but otherwise fairly vanilla, little different from, for example, the CrewAI semantics. Fortunately, the above introduction just scratched the surface of the motleycrew `Task` API.

Each crew is automatically given an embedded [knowledge graph backend](knowledge_graph.html). We currently use [kuzu](https://github.com/kuzudb/kuzu) because it's embeddable, available under an MIT license, and is one of the LlamaIndex-supported KG backends - please raise an issue on GitHub if you'd like us to support others.
The relationships between tasks are automatically stored in the KG backend; but the agents that are working on the tasks can also read and write any other context they want to share.

A `Task` object must implement only two methods: `get_next_unit()` and `get_worker()`. The former returns a data object (`TaskUnit`) describing the next part of the task to be done (or `None` if there is nothing to be done for that particular `Task` at the moment), and the latter returns the worker (typically an agent) that this data object can be given to for execution. The crew keeps querying all available `Task`s for `TaskUnits` and dispatching them, until done.

You can see how this dispatch method easily supports different execution backends, from synchronous to asyncio, threaded, etc.


### Example: Recursive question-answering in the research agent

An example of the power of this approach is the [research agent](examples/research_agent.html). It consists of two tasks, `QuestionTask` and `AnswerTask`. The `QuestionTask` starts with a user question, and embeds this into the graph as the first un-answered question. Its `get_next_unit()` method looks up all the as yet un-answered questions, and chooses the one that's most salient to the original question (so that question is the `TaskUnit` it returns). Its worker then retrieves the context (RAG-style) for that question, but instead of answering it, creates up to 3 further questions that would be most helpful to answer in order to answer the original question. We thus build up a tree of questions, where each non-leaf node has a retrieval context attached to it - all stored in the knowledge graph for easy retrieval. This goes on until we have enough questions (currently just a fixed number of iterations).

The `AnswerTask` then rolls the tree back up. It ignores all the questions without a retrieved context; and the `TaskUnit` that its `get_next_unit()` returns is then any question that has no un-answered children. Its worker then proceeds to answer that question using its retrieved context and the answers from its children, if any. This goes on until we've worked our way back up to answering the original question.



This shows how the tasks can create `TaskUnit`s for themselves and for each other, which enables a whole new level of self-organization. 

The different `Task`s don't have to all form part of a connected DAG either. For example, two tasks could take turns creating `TaskUnit`s for one another - just one of many interaction patterns possible within the architecture.